# 2 磨刀不误砍柴工：工具准备

# 2.5 语料库准备

In [ ]:
import nltk

### 2.5.3 准备《射雕》语料库

读入数据框；  
这里强制用csv格式读入，实际上文本中并没有'aaa'的数据分割符，所以读到内存中的结果只有一列，并命名为names=['txt']；  
此时的数据框只有一列（7165行），列名是'txt'；

In [ ]:
import pandas as pd
# 有的环境配置下read_table出错，因此改用read_csv
raw = pd.read_csv(r'./data/金庸-射雕英雄传txt精校版.txt',
                  names=['txt'], sep='aaa', encoding="GBK", engine='python')
print(len(raw))
# raw

加章节标示；  
raw['head'] = raw.txt.apply(f)：使用函数 $f$ 作用在txt一列中的每一个元素，将结果作为一个新的列放到'head'一列中；  

In [ ]:
# 章节判断用变量预处理
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")

raw['head'] = raw.txt.apply(m_head)
raw['mid'] = raw.txt.apply(m_mid)
raw['len'] = raw.txt.apply(len)
# raw.head(50)

raw.loc[i, 'chap'] = chapnum给chap一行中第i列赋值为i，给txt的每一列都加上章节索引号；

In [ ]:
# 章节判断
chapnum = 0
for i in range(len(raw)):
    if raw['head'][i] == "第" and raw['mid'][i] > 0 and raw['len'][i] < 30 :
        chapnum += 1
    if chapnum >= 40 and raw['txt'][i] == "附录一：成吉思汗家族" :
        chapnum = 0
    raw.loc[i, 'chap'] = chapnum
    
# 删除临时变量，这里必须删除，否则后续sum函数处会出错
del raw['head']
del raw['mid']
del raw['len']

提取章节信息


In [ ]:
raw[raw.chap == 2].head()
from matplotlib import pyplot as plt
%matplotlib inline
raw.txt.agg(len).plot.box()

rawgrp = raw.groupby('chap')
chapter = rawgrp.agg(sum) # 只有字符串列的情况下，sum函数自动转为合并字符串
chapter = chapter[chapter.index != 0]
chapter.txt[2]

# 分词

## 分词原理简介

词是中文表达语义的最小单位，自然语言处理的基础步骤就是分词，对于拉丁语系来说最基本的语意单位是单词，不存在分词问题，直接用空格即可对文本作出分割即可。对于亚洲语言来说，最基本的语意单位是单字，分词的结果对亚洲语意信息处理至为关键。本小节以汉语为代表介绍分词原理即工具使用。 
  
** 分词的基本思想只有两个：**
1. 基于词典的分词方法；
2. 基于统计的分词方法。

### 基于词典的分词方法

基于词典的分词方法首先会建立一个充分大的词典，然后依据一定的策略扫描句子，若句子中的某个子串与词典中的某个词匹配，则分词成功。  
常见的扫描策略有：正向最大匹配、逆向最大匹配、双向最大匹配和最少词数分词。  
  
**正向最大匹配**  
对输入的句子从左至右，以贪心的方式切分出当前位置上长度最大的词，组不了词的字单独划开。其分词原理是：词的颗粒度越大，所能表示的含义越精确。
  
**逆向最大匹配**  
原理与正向最大匹配相同，但顺序不是从首字开始，而是从末字开始，而且它使用的分词词典是逆序词典，其中每个词条都按逆序方式存放。在实际处理时，先将句子进行倒排处理，生成逆序句子，然后根据逆序词典，对逆序句子用正向最大匹配。
  
**双向最大匹配**  
将正向最大匹配与逆向最大匹配组合起来，对句子使用这两种方式进行扫描切分，如果两种分词方法得到的匹配结果相同，则认为分词正确，否则，按最小集处理。  
  
**最少词数分词**  
即一句话应该分成数量最少的词串，该方法首先会查找词典中最长的词，看是不是所要分词的句子的子串，如果是则切分，然后不断迭代以上步骤，每次都会在剩余的字符串中取最长的词进行分词，最后就可以得到最少的词数。  
  
**总结**  
基于词典的分词方法简单、速度快，效果也还可以，但对歧义和新词的处理不是很好，对词典中未登录的词没法进行处理。

### 基于统计的分词方法

基于统计的分词方法是从大量已经分词的文本中，利用统计学习方法来学习词的切分规律，从而实现对未知文本的切分。随着大规模语料库的建立，基于统计的分词方法不断受到研究和发展，渐渐成为了主流。  
常用的统计学习方法有：隐马尔可夫模型（HMM）、条件随机场（CRF）和基于深度学习的方法。
  
**HMM和CRF**  
这两种方法实质上是对序列进行标注，将分词问题转化为字的分类问题，每个字有4种词位（类别）：词首（B）、词中（M）、词尾（E）和单字成词（S）。由字构词的方法并不依赖于事先编制好的词典，只需对分好词的语料进行训练即可。当模型训练好后，就可对新句子进行预测，预测时会针对每个字生成不同的词位。其中HMM属于生成式模型，CRF属于判别式模型。  
  
**基于深度学习的方法**  
神经网络的序列标注算法在词性标注、命名实体识别等问题上取得了优秀的进展，这些端到端的方法也可以迁移到分词问题上。与所有深度学习的方法一样，该方法需要较大的训练语料才能体现优势，代表为BiLSTM-CRF。
  
**总结**  
基于统计的分词方法能很好地处理歧义和新词问题，效果比基于词典的要好，但该方法需要有大量人工标注分好词的语料作为支撑，训练开销大，就分词速度而言不如前一种。

### jieba分词原理

在实际应用中一般是将词典与统计学习方法结合起来，既发挥词典分词切分速度快的特点，又利用了统计分词结合上下文识别生词、自动消除歧义的优点。结巴分词正是这一类的代表，下面简要介绍一下它的实现算法。  
   
官方Github上对所用算法有以下三个方面：  
**基于前缀词典实现高效的词图扫描，生成句子中汉字所有可能成词情况所构成的有向无环图 (DAG)**  
- 构造前缀词典  
    结巴分词首先会依照统计词典dict.txt构造前缀词典。dict.txt含有近35万的词条，每个词条占用一行，其中每一行有3列，第一列为词条，第二列为对应的词频，第三列为词性，构造前缀词典需要用到前两列。  
    具体做法为：首先定义一个空的python字典，然后遍历dict.txt的每一行，取词条作为字典的键，词频作为对应的键值，然后遍历该词条的前缀，如果前缀对应的键不在字典里，就把该前缀设为字典新的键，对应的键值设为0，如果前缀在字典里，则什么都不做。
    这样等遍历完dict.txt后，前缀词典就构造好了。在构造前缀词典时，会对统计词典里所有词条的词频做一下累加，累加值等计算最大概率路径时会用到。  
**采用了动态规划查找最大概率路径, 找出基于词频的最大切分组合**  
**对于未登录词，采用了基于汉字成词能力的 HMM 模型，使用了 Viterbi 算法** 

## jieba分词基本用法

官方例子

In [ ]:
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list) + r' (此处，“杭研”并没有在词典中，但是也被Viterbi算法识别出来了)')

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

分词《射雕》语料

In [ ]:
tmpstr = "郭靖和哀牢山三十六剑。"
res1 = jieba.cut(tmpstr) # 精确模式
print(res1) # 是一个可迭代的 generator，可以使用 for 循环来遍历结果，本质上类似list
print('/'.join(res1))

## 修改字典

### 使用自定义词典
- 开发者可以指定自己自定义的词典，以便包含 jieba 词库里没有的词。虽然 jieba 有新词识别能力，但是自行添加新词可以保证更高的正确率
- 用法： jieba.load_userdict(file_name) # file_name 为文件类对象或自定义词典的路径
- 词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。
- 词频省略时使用自动计算的能保证分出该词的词频。
  
load_userdict(file_name)  
file_name：文件类对象或自定义词典的路径  
  
词典基本格式  
一个词占一行：词、词频（可省略）、词性（可省略），用空格隔开  
词典文件必须为 UTF-8 编码  
    必要时可以使用Uedit32进行文件编码转换  
云计算 5  
李小福 2 nr  
easy_install 3 eng  
台中  

In [ ]:
# 动态增加词典
jieba.add_word("哀牢山三十六剑")
print('/'.join(jieba.cut(tmpstr)))

# 动态删除词典
jieba.del_word("哀牢山三十六剑")
print('/'.join(jieba.cut(tmpstr)))

# dict为自定义词典的路径
dict = './data/金庸小说词库.txt'
jieba.load_userdict(dict) 
print('/'.join(jieba.cut(tmpstr)))

### 使用搜狗细胞词库

1. 使用搜狗细胞词库：
https://pinyin.sogou.com/dict/  
2. 按照词库分类或者关键词搜索方式，查找并下载所需词库；
3. 使用转换工具，将其转换为txt格式，工具为深蓝词库转换或奥创词库转换，然后在程序中导入相应词库。

## 去除停用词

**常见的停用词种类**。
1. 超高频的常用词：基本不携带有效信息/歧义太多无分析价值
的、地、得  
虚词：如介词，连词等  
只、条、件  
当、从、同  
2. 专业领域的高频词：基本不携带有效信息
3. 视情况而定的停用词
呵呵  
emoj  
  
**分词后去除停用词**  
基本步骤
1. 读入停用词表文件
2. 正常分词
3. 在分词结果中去除停用词
  
程序框架：新列表 = [ word for word in 源列表 if word not in 停用词列表 ]  
该方法存在的问题：停用词必须要被分词过程正确拆分出来才行。

In [ ]:
# 一个简单的例子
newlist = [ w for w in jieba.cut(tmpstr) if w not in ['和', '。'] ] 
print(newlist)

In [ ]:
# 一个通用的例子
import pandas as pd
tmpdf = pd.read_csv('./data/停用词.txt',
                    names = ['w'], sep = 'aaa', encoding = 'utf-8', engine='python')
# print(tmpdf.head())
# 熟悉Python的可以直接使用 open('stopWord.txt').readlines()获取停用词list，效率更高
[ w for w in jieba.cut(tmpstr) if w not in list(tmpdf.w) ] 

**用extract_tags函数去除停用词**。  
方法特点：  
&nbsp; 根据TF-IDF算法将特征词提取出来，在提取之前去掉停用词  
&nbsp; 可以人工指定停用词字典  
&nbsp; jieba.analyse.set_stop_words()  

In [ ]:
# 使用预先准备的停用词表
import jieba.analyse as ana
ana.set_stop_words('./data/停用词.txt')
jieba.lcut(tmpstr) # 读入的停用词列表对分词结果无效

In [ ]:
ana.extract_tags(tmpstr, topK = 20) # 使用TF-IDF算法提取关键词，并同时去掉停用词

## 词性标注

- jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。
- 标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。
- 除了jieba默认分词模式，提供paddle模式下的词性标注功能。paddle模式采用延迟加载方式，通过enable_paddle()安装paddlepaddle-tiny，并且import相关代码；
- 用法示例

In [ ]:
# 官方例子
import jieba
import jieba.posseg as pseg
words = pseg.cut("我爱北京天安门") #jieba默认模式
jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持
words = pseg.cut("我爱北京天安门",use_paddle=True) #paddle模式
for word, flag in words:
    print('%s %s' % (word, flag))

In [ ]:
# 《射雕》预料词性标注，
import jieba.posseg as psg

tmpres = psg.cut(tmpstr) # 附加词性的分词结果
print(tmpres)

for item in tmpres:
    print(item.word, item.flag)

- 不用paddle模式的结果： 
郭靖 nr  
和 c  
哀牢山 ns  
三十六 m  
剑 n  
。 x  
- 使用paddle模式的结果：  
郭靖 PER  
和 c  
哀牢山三十六剑。 nz  
  
结论：对比发现jieba分词使用paddle模式效果会好很多

## 实现《射雕》一书分词

使用停用词

In [ ]:
# 使用预先准备的停用词表
import jieba.analyse as ana
ana.set_stop_words('./data/停用词.txt')
# jieba.lcut(tmpstr) # 读入的停用词列表对分词结果无效

In [ ]:
ana.extract_tags(tmpstr, topK = 20) # 使用TF-IDF算法提取关键词，并同时去掉停用词

In [ ]:
#分词
word_list = jieba.lcut(chapter.txt[1])
word_list[:100]

In [ ]:
word_list = jieba.lcut(" ".join(raw.txt))
df = pd.DataFrame(word_list, columns = ['word'])
df.head(30)

In [ ]:
result = df.groupby(['word']).size()
print(type(result))
freqlist = result.sort_values(ascending=False)
freqlist[:20]

In [ ]:
fdist = nltk.FreqDist(word_list) # 生成完整的词条频数字典
fdist

In [ ]:
import wordcloud
myfont = r'/System/Library/fonts/PingFang.ttc'
text = 'this is shanghai, 郭靖, 和, 哀牢山 三十六剑'
cloudobj = wordcloud.WordCloud(font_path=myfont).generate(text)  
print(cloudobj)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(cloudobj)
plt.axis("off")
plt.show()

In [ ]:
# 更改词云参数设定
cloudobj = wordcloud.WordCloud(font_path = myfont, 
    width = 360, height = 180,
    mode = "RGBA", background_color = None).generate(text)  

plt.imshow(cloudobj)
plt.axis("off")
plt.show()

生成射雕第一章词云

In [ ]:
import pandas as pd
import jieba

stoplist = list(pd.read_csv('./data/停用词.txt', names = ['w'], sep = 'aaa', 
                            encoding = 'utf-8', engine='python').w)

def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) if w not in stoplist ] 

cloudobj = wordcloud.WordCloud(font_path = myfont, 
    width = 1200, height = 800,
    mode = "RGBA", background_color = None,
    stopwords = stoplist).generate(' '.join(jieba.lcut(chapter.txt[1])))  

plt.imshow(cloudobj)
plt.axis("off")
plt.show()

In [ ]:
from gensim.corpora import Dictionary

texts = [['human', 'interface', 'computer']]
dct = Dictionary(texts)  # fit dictionary
dct.token2id

In [ ]:
chapter.head()

In [ ]:
# 设定分词及清理停用词函数
# 熟悉Python的可以使用 open('stopWord.txt').readlines（） 获取停用词list，效率更高
stoplist = list(pd.read_csv('./data/停用词.txt', names = ['w'], sep = 'aaa', 
                            encoding = 'utf-8', engine='python').w)
import jieba 
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) 
            if w not in stoplist and len(w) > 1 ] 

## 参考文献

1. 中文分词概述及结巴分词原理 https://www.cnblogs.com/cyandn/p/10891608.html；

# 文档相似度

## 基本概念

## 词条相似度：word2vec

导入需要的包

In [ ]:
import jieba

检查一下数据信息

In [ ]:
chapter.txt[1]
# 分词和预处理，生成list of list格式
import jieba

chapter['cut'] = chapter.txt.apply(jieba.lcut)
chapter.head()

In [ ]:
# 初始化word2vec模型和词表
from gensim.models.word2vec import Word2Vec

n_dim = 300 # 指定向量维度，大样本量时300~500较好

w2vmodel = Word2Vec(size = n_dim, min_count = 10)
w2vmodel.build_vocab(chapter.cut) # 生成词表
w2vmodel

In [ ]:
# 在评论训练集上建模（大数据集时可能会花费几分钟）
# 本例消耗内存较少
%time w2vmodel.train(chapter.cut, \
               total_examples = w2vmodel.corpus_count, epochs = 10)

In [ ]:
# 训练完毕的模型实质
print(w2vmodel.wv["郭靖"].shape)
w2vmodel.wv["郭靖"]

In [ ]:
w2vmodel.wv.most_similar("郭靖")

## 文档相似度：词袋模型计算

基于词袋模型计算

In [ ]:
cleanchap = [ " ".join(m_cut(w)) for w in chapter.txt.iloc[:5]] 

from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer() 

resmtx = countvec.fit_transform(cleanchap)
resmtx

## doc2vec

# 文本分类

本专题首先概述文本分类的概念、思路，随后用多个demo说明文本分类的流程及工具的使用。

## 文本分类的概念

文本分类（text classification），指的是将一个文档归类到一个或多个类别的自然语言处理任务。文本分类的应用场景非常广泛，包括垃圾邮件过滤、自动打标等任何需要自动归档文本的场合。  
文本分类在机器学习中属于监督学习，其流程是：人工标注文档类别、利用语料训练模型、利用模型训练文档的类别。


## 文本的特征提取

在向量空间模型中，表示文本的特征项可以选择文字、词、短语、甚至“概念”等多种元素，目前常用的特征提取方法有：基于文档频率的特征提取法、信息增益法、$\chi^2$统计量法、互信息法等。

## 分类器的选择和训练

理论上讲，在文本特征抽取之后，就进入了常规机器学习分类模型的框架，但作为文本分类也有其特殊性，主要有以下几点：
    1. 自变量（词条）数量极多；
    2. 各自变量之间（词条）不可能完全独立；
    3. 大部分自变量（词条）都是干扰项，对分类没有贡献；
所以在分类模型选择上主要考虑以下几点：
    1. 速度-文本数据量一般比较大；
    2. 变量筛选能力-能够从大部分是无效变量的情况下筛选出有效变量；
    3. 容错性-分类模型是建立在特征抽取的基础上，特征抽取过程本身不可避免的带来部分信息差错；
    4. 共线容忍度-词条之间不可能相互独立，很多模型都有变量的独立性假设。
基于上面两个方面的考虑，文本分类的模型通常使用朴素贝叶斯、svm两个模型。

## 标准化评测

评测指标通常是四项：  
$P = \frac{TP}{TP + FP}$：精确率  
$R = \frac{TP}{TP + FN}$：召回率  
$F_1 = \frac{2*P*R}{P + R}$：调和平均  
running_time：训练时间

## demo1-搜狗实验室数据分类

这个实验使用HanLP这个自然语言处理库。

### 准备文本分类语料库
1. ensure_data第一次执行时下载 http://file.hankcs.com/corpus/sogou-text-classification-corpus-mini.zip 到 /Users/kitty/anaconda3/envs/nlp/lib/python3.6/site-packages/pyhanlp/static/data/test/搜狗文本分类语料库迷你版.zip，下载完成后自动解压zip包并删除zip包；
2. 搜狗实验室提供的这个语料共分成教育,汽车,健康,军事,体育五类，每一类一个文件夹，每类（每个文件夹）1000个文档；

In [5]:
from pyhanlp import *
from tests.test_utility import ensure_data

sogou_corpus_path = ensure_data('搜狗文本分类语料库迷你版',
                                'http://file.hankcs.com/corpus/sogou-text-classification-corpus-mini.zip')

1. dataSet.load(sogou_corpus_path)将数据加载到内存中;

In [ ]:
MemoryDataSet = JClass('com.hankcs.hanlp.classification.corpus.MemoryDataSet')
dataSet = MemoryDataSet()  # 将数据集加载到内存中
dataSet.load(sogou_corpus_path)  # 加载data/test/搜狗文本分类语料库迷你版
allClasses = dataSet.getCatalog().getCategories()  # 获取标注集
print("标注集：%s" % (allClasses))
for document in dataSet.iterator():
    print("第一篇文档的类别：" + allClasses.get(document.category))
    break

### 文档的特征提取

在本demo中，使用词袋（bag of words，简称BW）模型刻画文档特征。词袋是信息检索与自然语言处理中最常用的文档表示模型。在具体训练词袋模型时，常用的有以下四种方式作为统计指标：
1. 分词后每个词的词频，词频向量适合主题较多的数据集；
2. 布尔词频：词语存在为1，不存在为0，布尔词频适合长度较短的数据集；
3. TF-IDF：将每个词语的倒排频次也纳入考虑，TF-IDF适合主题较少的数据集；
4. 词向量，适合OOV问题严重的数据集；
在HanLP中，使用的是TF（即第一种统计指标）作为词袋模型的统计指标，并且考虑到常用单词对分类决策贡献不大，也有可能有一些单词在每篇文档中均匀出现，为了消除这些影响，HanLP一方面使用停用此表，另一方面使用卡方非参数检验（Chi-square test），在真正构建词袋模型前对语料进行处理，解决上面存在的问题，然后再训练词袋模型。  

### 朴素贝叶斯分类器

1. SafeJClass：safe java class，从字面意思上可以看出，这里是在python中使用Java中的类，因为HanLP核心使用Java开发的；

In [ ]:
NaiveBayesClassifier = SafeJClass('com.hankcs.hanlp.classification.classifiers.NaiveBayesClassifier')
IOUtil = SafeJClass('com.hankcs.hanlp.corpus.io.IOUtil')

1. classifier.train(sogou_corpus_path)，这么一句代码执行后，模型训练就完成了！这其中的过程包括：使用分词器分词->使用卡方特征选择->使用词袋模型（词频作为统计量）向量话文本数据->喂给贝叶斯分类器训练。这些操作流程在这句代码中都完成了！
2. classifier = NaiveBayesClassifier()，构建并初始化一个贝叶斯分类器对象，对象中的方法包括分词、卡方检验、词袋模型训练、朴素贝叶斯训练算法等。
3. classifier.train(sogou_corpus_path)，把数据喂给贝叶斯分类器进行训练；

In [ ]:
def train_or_load_classifier():
    model_path = sogou_corpus_path + '.ser'
    if os.path.isfile(model_path):
        return NaiveBayesClassifier(IOUtil.readObjectFrom(model_path))
    classifier = NaiveBayesClassifier()
    classifier.train(sogou_corpus_path)
    model = classifier.getModel()
    IOUtil.saveObjectTo(model, model_path)
    return NaiveBayesClassifier(model)

1. 定义一个通用预测函数，格式化输出feature、label；

In [ ]:
def predict(classifier, text):
    print("《%16s》\t属于分类\t【%s】" % (text, classifier.classify(text)))

1. 用每个类别中比较明显的文本内容进行测试；

In [ ]:
classifier = train_or_load_classifier()
predict(classifier, "C罗获2018环球足球奖最佳球员 德尚荣膺最佳教练")
predict(classifier, "英国造航母耗时8年仍未服役 被中国速度远远甩在身后")
predict(classifier, "研究生考录模式亟待进一步专业化")
predict(classifier, "如果真想用食物解压,建议可以食用燕麦")
predict(classifier, "通用及其部分竞争对手目前正在考虑解决库存问题")

### 支持向量机分类器

朴素贝叶斯模型比较简单，由于其对条件概率分布做了条件独立性假设，这是一个较强的假设，冠以“朴素”由此而来，其带来的问题有时会影响分类的准确性。接下来使用SVM算法，这是传统机器学习中一个集大成模型。接下来的代码和朴素贝叶斯套路一样，具体细节不再解释。

1. 加载LinearSVMClassifier类；

In [ ]:
LinearSVMClassifier = SafeJClass('com.hankcs.hanlp.classification.classifiers.LinearSVMClassifier')
IOUtil = SafeJClass('com.hankcs.hanlp.corpus.io.IOUtil')

1. SVM训练数据

In [ ]:
def train_or_load_classifier():
    model_path = sogou_corpus_path + '.svm.ser'
    if os.path.isfile(model_path):
        return LinearSVMClassifier(IOUtil.readObjectFrom(model_path))
    classifier = LinearSVMClassifier()
    classifier.train(sogou_corpus_path)
    model = classifier.getModel()
    IOUtil.saveObjectTo(model, model_path)
    return LinearSVMClassifier(model)

1. 用每个类别中比较明显的文本内容进行测试；

In [ ]:
classifier = train_or_load_classifier()
predict(classifier, "C罗获2018环球足球奖最佳球员 德尚荣膺最佳教练")
predict(classifier, "潜艇具有很强的战略威慑能力与实战能力")
predict(classifier, "研究生考录模式亟待进一步专业化")
predict(classifier, "如果真想用食物解压,建议可以食用燕麦")
predict(classifier, "通用及其部分竞争对手目前正在考虑解决库存问题")

### 各种算法组合评测对比

1. IClassifier
2. NaiveBayesClassifier：朴素贝叶斯分类器类；
3. LinearSVMClassifier：线性SVM分类器类；
4. FileDataSet：用于数据预处理，可以设置分词器、train/test比例
5. IDataSet
6. MemoryDataSet
7. Evaluator
8. FMeasure
9. BigramTokenizer：二元语法
10. HanLPTokenizer：中文分词
11. ITokenizer

In [ ]:
IClassifier = JClass('com.hankcs.hanlp.classification.classifiers.IClassifier')
NaiveBayesClassifier = JClass('com.hankcs.hanlp.classification.classifiers.NaiveBayesClassifier')
LinearSVMClassifier = JClass('com.hankcs.hanlp.classification.classifiers.LinearSVMClassifier')
FileDataSet = JClass('com.hankcs.hanlp.classification.corpus.FileDataSet')
IDataSet = JClass('com.hankcs.hanlp.classification.corpus.IDataSet')
MemoryDataSet = JClass('com.hankcs.hanlp.classification.corpus.MemoryDataSet')
Evaluator = JClass('com.hankcs.hanlp.classification.statistics.evaluations.Evaluator')
FMeasure = JClass('com.hankcs.hanlp.classification.statistics.evaluations.FMeasure')
BigramTokenizer = JClass('com.hankcs.hanlp.classification.tokenizers.BigramTokenizer')
HanLPTokenizer = JClass('com.hankcs.hanlp.classification.tokenizers.HanLPTokenizer')
ITokenizer = JClass('com.hankcs.hanlp.classification.tokenizers.ITokenizer')

1. training_corpus = FileDataSet().setTokenizer(tokenizer).load(sogou_corpus_path, "UTF-8", 0.9)的输出是：  
模式:训练集  
文本编码:UTF-8  
根目录:/Users/kitty/Work/Projects/hhnlp/pyhanlp-master/pyhanlp/static/data/test/搜狗文本分类语料库迷你版  
加载中...  
[教育]...100.00% 900 篇文档  
[汽车]...100.00% 900 篇文档  
[健康]...100.00% 900 篇文档  
[军事]...100.00% 900 篇文档  
[体育]...100.00% 900 篇文档  
耗时 27834 ms 加载了 5 个类目,共 4500 篇文档  
在此过程首先加载相应比例的数据到内存，然后使用相应的分词器进行分词，并保存在training_corpus对象中；  
2. classifier.train(training_corpus)的输出：
原始数据集大小:4500  
使用卡方检测选择特征中...耗时 6199 ms,选中特征数:19691 / 97771 = 20.14%  
贝叶斯统计结束  
在此过程中进行了卡方特征选择、词袋模型训练、贝叶斯模型训练；
3. testing_corpus = MemoryDataSet(classifier.getModel()).load(sogou_corpus_path, "UTF-8", -0.1)的输出：  
模式:测试集  
文本编码:UTF-8  
根目录:/Users/kitty/Work/Projects/hhnlp/pyhanlp-master/pyhanlp/static/data/test/搜狗文本分类语料库迷你版  
加载中...  
[教育]...100.00% 100 篇文档  
[汽车]...100.00% 100 篇文档  
[健康]...100.00% 100 篇文档  
[军事]...100.00% 100 篇文档  
[体育]...100.00% 100 篇文档  
耗时 870 ms 加载了 5 个类目,共 500 篇文档  
这一步，加载测试集后，模型对测试集进行了预测。
4. result = Evaluator.evaluate(classifier, testing_corpus)  
 P	     R	    F1	     A	        
 27.32	 53.00	 36.05	 62.40	教育  
 88.89	 16.00	 27.12	 82.80	汽车  
 19.51	 40.00	 26.23	 55.00	健康  
 14.71	  5.00	  7.46	 75.20	军事  
 16.33	  8.00	 10.74	 73.40	体育  
 33.35	 24.40	 28.18	 24.40	avg.  
data size = 500, speed = 7692.31 doc/s  


In [ ]:
def evaluate(classifier, tokenizer):
    training_corpus = FileDataSet().setTokenizer(tokenizer).load(sogou_corpus_path, "UTF-8", 0.9)
    classifier.train(training_corpus)
    testing_corpus = MemoryDataSet(classifier.getModel()).load(sogou_corpus_path, "UTF-8", -0.1)
    result = Evaluator.evaluate(classifier, testing_corpus)
    print(classifier.getClass().getSimpleName() + "+" + tokenizer.getClass().getSimpleName())
    print(result)

In [ ]:
evaluate(NaiveBayesClassifier(), HanLPTokenizer())
evaluate(NaiveBayesClassifier(), BigramTokenizer())
evaluate(LinearSVMClassifier(), HanLPTokenizer())
evaluate(LinearSVMClassifier(), BigramTokenizer())

## 使用HanLP进行文本分类的一般步骤

本小节总结上一节中的分析过程，提炼出使用HanLP进行文本的操作步骤。

### 准备文本分类语料库

数据的目录结构如下：  
搜狗门户数据（汽车（1000个txt文档）、教育（1000个txt文档）、健康（1000个txt文档）、军事（1000个txt文档）、体育（1000个txt文档））

In [14]:
from pyhanlp import *
corpus_path = r'/Users/kitty/Work/Projects/text_mining/data/搜狗文本分类语料库迷你版'
# 把数据加载到内存中，查看数据属性
MemoryDataSet = JClass('com.hankcs.hanlp.classification.corpus.MemoryDataSet')
dataSet = MemoryDataSet()  # 将数据集加载到内存中
dataSet.load(corpus_path)  # 加载data/test/搜狗文本分类语料库迷你版
allClasses = dataSet.getCatalog().getCategories()  # 获取标注集
print("标注集：%s" % (allClasses))
for document in dataSet.iterator():
    print("第一篇文档的类别：" + allClasses.get(document.category))
    break

标注集：[教育, 汽车, 健康, 军事, 体育]
第一篇文档的类别：教育


### 准备分词器

1. 因为HanLP是面向对象的，不是面向过程的，所以所有的操作方法根据继承关系最后都会加诸到一个具体的对象上，只需要把相应类准备好即可；
2. BigramTokenizer，中文文本，将相邻字符作为二元语法输出，采用的是2016年An Efficient Chinese Text Classifier这篇论文结果，这篇论文结论是对于中文，不分词反而取得更好的分类效率，代价是增加了模型训练的复杂度；
3. HanLPTokenizer，HanLP预置的Tokenizer，用于中文文本分词；
4. BlankTokenizer，英文文本，使用空格分词；

In [15]:
BigramTokenizer = JClass('com.hankcs.hanlp.classification.tokenizers.BigramTokenizer')
HanLPTokenizer = JClass('com.hankcs.hanlp.classification.tokenizers.HanLPTokenizer')
BlankTokenizer = JClass('com.hankcs.hanlp.classification.tokenizers.BlankTokenizer')
tokenizer = BigramTokenizer()

### 准备训练集

In [17]:
FileDataSet = JClass('com.hankcs.hanlp.classification.corpus.FileDataSet')
training_corpus = FileDataSet().setTokenizer(tokenizer).load(corpus_path, "UTF-8", 0.9)

### 卡方特征选择

这个方法和相应的参数都被隐藏的模型中，用户只需知道分词后需要进行卡方特征选择。

### 准备分类器

In [18]:
NaiveBayesClassifier = JClass('com.hankcs.hanlp.classification.classifiers.NaiveBayesClassifier')
LinearSVMClassifier = JClass('com.hankcs.hanlp.classification.classifiers.LinearSVMClassifier')
model_class = LinearSVMClassifier

### 训练模型

In [19]:
IOUtil = SafeJClass('com.hankcs.hanlp.corpus.io.IOUtil')
model_path = r'/Users/kitty/anaconda3/envs/nlp/lib/python3.6/site-packages/pyhanlp/static/data/test/'
def train_or_load_classifier(model_class, model_path, training_corpus):
    classifier = model_class()
    model_path += classifier.getClass().getSimpleName() + '.ser'
    if os.path.isfile(model_path):
        print(model_path)
        return model_class(IOUtil.readObjectFrom(model_path))
    classifier.train(training_corpus)
    model = classifier.getModel()
    IOUtil.saveObjectTo(model, model_path)
    return model_class(model)

classifier = train_or_load_classifier(model_class, model_path, training_corpus)

### 评测训练效果

In [13]:
Evaluator = JClass('com.hankcs.hanlp.classification.statistics.evaluations.Evaluator')

def evaluate(classifier, corpus_path, tokenizer):
    testing_corpus = MemoryDataSet(classifier.getModel()).load(corpus_path, "UTF-8", -0.1)
    result = Evaluator.evaluate(classifier, testing_corpus)
    print(classifier.getClass().getSimpleName() + "+" + tokenizer.getClass().getSimpleName())
    print(result)

evaluate(classifier, corpus_path, tokenizer)

LinearSVMClassifier+BigramTokenizer
     P	     R	    F1	     A	      
 94.17	 97.00	 95.57	 98.20	教育
 99.00	 99.00	 99.00	 99.60	汽车
 98.98	 97.00	 97.98	 99.20	健康
 98.02	 99.00	 98.51	 99.40	军事
100.00	 98.00	 98.99	 99.60	体育
 98.03	 98.00	 98.02	 98.00	avg.
data size = 500, speed = 16666.67 doc/s



In [ ]:
LinearSVMClassifier+BigramTokenizer
     P	     R	    F1	     A	      
 94.17	 97.00	 95.57	 98.20	教育
 99.00	 99.00	 99.00	 99.60	汽车
 98.98	 97.00	 97.98	 99.20	健康
 98.02	 99.00	 98.51	 99.40	军事
100.00	 98.00	 98.99	 99.60	体育
 98.03	 98.00	 98.02	 98.00	avg.
data size = 500, speed = 13513.51 doc/s